In [1]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 7.8 MB/s 


In [2]:
import numpy as np

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

In [3]:
# Parameters

crude_numbers = range(1,2+1)
petrols = ["Premium_fuel", "Regular_fuel"]
end_product_names = ["Premium_fuel", "Regular_fuel", "Jet_fuel", "Fuel_oil", "Lube_oil"]
distillation_products_names = ["Light_naphtha", "Medium_naphtha", "Heavy_naphtha",
                               "Light_oil", "Heavy_oil", "Residuum"]
naphthas = ["Light_naphtha", "Medium_naphtha", "Heavy_naphtha"]
intermediate_oils = ["Light_oil", "Heavy_oil"]
cracking_products_names = ["Cracked_gasoline", "Cracked_oil"]
used_for_motor_fuel_names = ["Light_naphtha", "Medium_naphtha", "Heavy_naphtha",
                             "Reformed_gasoline", "Cracked_gasoline"]
used_for_jet_fuel_names = ["Light_oil", "Heavy_oil", "Residuum", "Cracked_oil"]

buy_limit = {1:20000, 2:30000}
lbo_min = 500
lbo_max = 1000

distill_cap = 45000
reform_cap = 10000
crack_cap = 8000

distillation_splitting_coefficients = {"Light_naphtha": (0.1, 0.15),
                          "Medium_naphtha": (0.2, 0.25),
                         "Heavy_naphtha": (0.2, 0.18),
                         "Light_oil": (0.12, 0.08),
                         "Heavy_oil": (0.2, 0.19),
                         "Residuum": (0.13, 0.12)}

cracking_splitting_coefficients = {("Light_oil","Cracked_oil"): 0.68,
                                   ("Heavy_oil","Cracked_oil"): 0.75,
                                   ("Light_oil","Cracked_gasoline"): 0.28,
                                   ("Heavy_oil","Cracked_gasoline"): 0.2}

reforming_splitting_coefficients = {"Light_naphtha": 0.6, "Medium_naphtha":0.52, "Heavy_naphtha":0.45}
end_product_profit = {"Premium_fuel":7, "Regular_fuel":6, "Jet_fuel":4, "Fuel_oil":3.5, "Lube_oil":1.5}
blending_coefficients = {"Light_oil": 0.55, "Heavy_oil": 0.17, "Cracked_oil": 0.22, "Residuum": 0.055}

lube_oil_factor = 0.5
pmf_rmf_ratio = 0.4

octance_number_coefficients = {
    "Light_naphtha":90,
    "Medium_naphtha":80,
    "Heavy_naphtha":70,
    "Reformed_gasoline":115,
    "Cracked_gasoline":105,
}
octance_number_fuel = {"Premium_fuel": 94,"Regular_fuel": 84}

vapor_pressure_constants = [0.6, 1.5, 0.05]


In [4]:
refinery = gp.Model('Refinery_Optimization')

# Variables
crudes = refinery.addVars(crude_numbers, ub=buy_limit, name="cr")    
end_products = refinery.addVars(end_product_names, name="end_prod")
end_products["Lube_oil"].lb= lbo_min
end_products["Lube_oil"].ub= lbo_max
distillation_products = refinery.addVars(distillation_products_names, name="dist_prod")
reform_usage = refinery.addVars(naphthas, name="napthas_to_reformed_gasoline")
reformed_gasoline = refinery.addVar(name="reformed_gasoline")
cracking_usage = refinery.addVars(intermediate_oils,name="intermediate_oils_to_cracked_gasoline")
cracking_products = refinery.addVars(cracking_products_names,  name="cracking_prods")
used_for_regular_motor_fuel = refinery.addVars(used_for_motor_fuel_names, name="motor_fuel_to_regular_motor_fuel")
used_for_premium_motor_fuel = refinery.addVars(used_for_motor_fuel_names, name="motot_fuel_to_premium_motor_fuel")
used_for_jet_fuel = refinery.addVars(used_for_jet_fuel_names, name="jet_fuel")
used_for_lube_oil = refinery.addVar(vtype=GRB.CONTINUOUS,name="residuum_used_for_lube_oil")


Restricted license - for non-production use only - expires 2024-10-28


In [5]:
# 1. Distillation capacity
DistillationCap = refinery.addConstr(crudes.sum() <= distill_cap, "Distill_cap")

In [7]:
# 2. Reforming capacity
ReformingCap = refinery.addConstr(reform_usage.sum() <= reform_cap, "Reform_cap")

In [8]:
# 3. Cracking capacity
CrackingCap = refinery.addConstr(cracking_usage.sum() <= crack_cap, "Crack_cap")

In [9]:
# 4.1-4.6 Yield (Crude oil products)
YieldCrudeOil = refinery.addConstrs((gp.quicksum(distillation_splitting_coefficients[dpn][crude-1]*crudes[crude] for crude in crudes)
                  == distillation_products[dpn] for dpn in distillation_products_names), "Splitting_distillation")

In [10]:
# 4.7 Yield (Reforming of Naphthas)
YieldNaphthas = refinery.addConstr(reform_usage.prod(reforming_splitting_coefficients) == reformed_gasoline, "Splitting_reforming")

In [11]:
# 4.8-4.9 Yield (Cracking of oils)
YieldCrackingOil = refinery.addConstrs((gp.quicksum(cracking_splitting_coefficients[oil, crack_prod]*cracking_usage[oil]
                           for oil in intermediate_oils) == cracking_products[crack_prod]
                  for crack_prod in cracking_products_names),
                 name="Splitting_cracking")

In [12]:
# 4.10 Yield (Lube oil)
YieldLubeOil = refinery.addConstr(lube_oil_factor*used_for_lube_oil == end_products["Lube_oil"],
                "Splitting_lube_oil")


In [13]:
# 4.11 Yield (Premium gasoline)
YieldPremium = refinery.addConstr(used_for_premium_motor_fuel.sum() == end_products["Premium_fuel"], "Blending_premium_fuel")

# 4.12 Yield (Regular gasoline)
YieldRegular = refinery.addConstr(used_for_regular_motor_fuel.sum() == end_products["Regular_fuel"], "Blending_regular_fuel")

# 4.13 Yield (Jet fuel)
YieldJetFuel = refinery.addConstr(used_for_jet_fuel.sum() == end_products["Jet_fuel"], "Continuity_jet_fuel")

In [14]:
# 5.1-5.3 Mass conservation (Naphthas)
MassBalNaphthas = refinery.addConstrs((reform_usage[naphtha] +
                    used_for_regular_motor_fuel[naphtha] +
                    used_for_premium_motor_fuel[naphtha] ==
                    distillation_products[naphtha] for naphtha in naphthas), "Continuity_napththa")


In [15]:
# 5.4 Mass Conservation (Light oil)
MassBalLightOil = refinery.addConstr(cracking_usage["Light_oil"]+
                used_for_jet_fuel["Light_oil"]+
                blending_coefficients["Light_oil"]*end_products["Fuel_oil"] ==
                distillation_products["Light_oil"], "Fixed_proportion_light_oil_for_blending")

# 5.5 Mass Conservation (Heavy oil)
MassBalHeavyOil = refinery.addConstr(cracking_usage["Heavy_oil"]+
                used_for_jet_fuel["Heavy_oil"]+
                blending_coefficients["Heavy_oil"]*end_products["Fuel_oil"] ==
                distillation_products["Heavy_oil"], "Fixed_proportion_heavy_oil_for_blending")

# 5.6 Mass Conservation (Cracked oil)
MassBalCrackedOil = refinery.addConstr(used_for_jet_fuel["Cracked_oil"]+
                blending_coefficients["Cracked_oil"]*end_products["Fuel_oil"] ==
                cracking_products["Cracked_oil"], "Fixed_proportion_cracked_oil_for_blending")

# 5.7 Mass Conservation (Residuum)
MassBalResiduum = refinery.addConstr(used_for_lube_oil +
                used_for_jet_fuel["Residuum"]+
                blending_coefficients["Residuum"]*end_products["Fuel_oil"] ==
                distillation_products["Residuum"], "Fixed_proportion_residuum_for_blending")

# 5.8 Mass conservation (Cracked gasoline)
MassBalCrackedGas = refinery.addConstr(used_for_regular_motor_fuel["Cracked_gasoline"] +
                used_for_premium_motor_fuel["Cracked_gasoline"] ==
                cracking_products["Cracked_gasoline"], "Continuity_cracked_gasoline")

# 5.9 Mass conservation (Reformed gasoline)
MassBalReformedGas = refinery.addConstr(used_for_regular_motor_fuel["Reformed_gasoline"] +
                used_for_premium_motor_fuel["Reformed_gasoline"] ==
                reformed_gasoline, "Continuity_reformed_gasoline")

In [16]:
# 7. Premium-to-regular proportion
Premium2Regular = refinery.addConstr(end_products["Premium_fuel"] >= pmf_rmf_ratio*end_products["Regular_fuel"],
                "Prem2reg_prop")



In [17]:
# 8.1-8.2 Octane tolerance
OctaneRegular = refinery.addConstr(used_for_regular_motor_fuel.prod(octance_number_coefficients) >=
                octance_number_fuel["Regular_fuel"] * end_products["Regular_fuel"],
                "Octane_tol_regular_fuel")

OctanePremium = refinery.addConstr(used_for_premium_motor_fuel.prod(octance_number_coefficients) >=
                octance_number_fuel["Premium_fuel"] * end_products["Premium_fuel"],
                "Octane_tol_premium_fuel")


In [18]:
# 9. Vapor-pressure tolerance
VaporPressure = refinery.addConstr(used_for_jet_fuel["Light_oil"] +
                vapor_pressure_constants[0]*used_for_jet_fuel["Heavy_oil"] +
                vapor_pressure_constants[1]*used_for_jet_fuel["Cracked_oil"] +
                vapor_pressure_constants[2]*used_for_jet_fuel["Residuum"] <= end_products["Jet_fuel"],
                "Vapor_pressure_tol")



In [19]:
# 0. Profit
refinery.setObjective(end_products.prod(end_product_profit), GRB.MAXIMIZE)

In [20]:
refinery.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 30 rows, 36 columns and 108 nonzeros
Model fingerprint: 0x6fbfe8a7
Coefficient statistics:
  Matrix range     [5e-02, 1e+02]
  Objective range  [2e+00, 7e+00]
  Bounds range     [5e+02, 3e+04]
  RHS range        [8e+03, 4e+04]
Presolve removed 14 rows and 14 columns
Presolve time: 0.02s
Presolved: 16 rows, 22 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1887574e+06   6.045565e+04   0.000000e+00      0s
      14    2.1136513e+05   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.03 seconds (0.00 work units)
Optimal objective  2.113651348e+05


In [21]:
for var in refinery.getVars():
    if abs(var.x) > 1e-6:
        print("{0} = {1}".format(var.varName, np.round(var.x, 2)))


cr[1] = 15000.0
cr[2] = 30000.0
end_prod[Premium_fuel] = 6817.78
end_prod[Regular_fuel] = 17044.45
end_prod[Jet_fuel] = 15156.0
end_prod[Lube_oil] = 500.0
dist_prod[Light_naphtha] = 6000.0
dist_prod[Medium_naphtha] = 10500.0
dist_prod[Heavy_naphtha] = 8400.0
dist_prod[Light_oil] = 4200.0
dist_prod[Heavy_oil] = 8700.0
dist_prod[Residuum] = 5550.0
napthas_to_reformed_gasoline[Heavy_naphtha] = 5406.86
reformed_gasoline = 2433.09
intermediate_oils_to_cracked_gasoline[Light_oil] = 4200.0
intermediate_oils_to_cracked_gasoline[Heavy_oil] = 3800.0
cracking_prods[Cracked_gasoline] = 1936.0
cracking_prods[Cracked_oil] = 5706.0
motor_fuel_to_regular_motor_fuel[Light_naphtha] = 273.07
motor_fuel_to_regular_motor_fuel[Medium_naphtha] = 10500.0
motor_fuel_to_regular_motor_fuel[Heavy_naphtha] = 2993.14
motor_fuel_to_regular_motor_fuel[Reformed_gasoline] = 1342.24
motor_fuel_to_regular_motor_fuel[Cracked_gasoline] = 1936.0
motot_fuel_to_premium_motor_fuel[Light_naphtha] = 5726.93
motot_fuel_to_premium